In [2]:
import os
import pandas as pd
from config_file import data_path

In [3]:
df_sessions = pd.read_csv(os.path.join(data_path, 'timestamp_track_event_type.csv'), parse_dates=True)
df_sessions.head()
df_sessions.dtypes

FileNotFoundError: [Errno 2] No such file or directory: '../data_v3/timestamp_track_event_type.csv'

In [ ]:
df_sessions.drop(["Unnamed: 0"], axis=1, inplace=True)
df_sessions.head()

In [ ]:
df_sessions["timestamp"] = pd.to_datetime(df_sessions["timestamp"], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
df_sessions.head()

In [ ]:
df_sessions["next_event"] =  (
    df_sessions["timestamp"].shift(-1) - df_sessions["timestamp"]
).dt.total_seconds()
df_sessions.head()

In [ ]:
df_sessions = df_sessions[df_sessions["event_type"] == "play"]
df_sessions.head()

In [ ]:
df_sessions.drop(["event_type"], axis=1, inplace=True)
df_sessions.head()

In [ ]:
df_artist_track = pd.read_csv(os.path.join(data_path, 'track_artist_duration.csv'))
df_artist_track.head()

In [ ]:
df_artist_track["duration_ms"] = df_artist_track["duration_ms"] / 1000
df_artist_track.rename(columns={"duration_ms": "duration_s", "id": "track_id"}, inplace=True)
df_artist_track.head()

In [ ]:
df_sessions = df_sessions.merge(df_artist_track, on="track_id", how="left")
df_sessions.head()

In [ ]:
df_sessions.drop(["track_id"], axis=1, inplace=True)
df_sessions.head()

In [ ]:
df_sessions["song_duration"] = df_sessions[['duration_s', 'next_event']].min(axis=1)
df_sessions.head()

In [ ]:
df_sessions[df_sessions["next_event"] >= 0].count(), df_sessions[df_sessions["next_event"] < 0].count() 

In [ ]:
df_sessions_filtered = df_sessions[df_sessions["next_event"] >= 0]
df_sessions_filtered.head()

In [ ]:
from datetime import datetime, timedelta
DAY_OF_PREDICTION = datetime(day=30, month=10, year=2023)
PERIOD_IN_DAYS = 30

start_date = DAY_OF_PREDICTION - timedelta(days=PERIOD_IN_DAYS)
df_sessions_filtered = df_sessions_filtered[df_sessions_filtered["timestamp"] >= start_date]
df_sessions_filtered.head()

In [ ]:
df_sessions_filtered.shape

In [ ]:
avg_song_duration = df_sessions_filtered.groupby("id_artist")["song_duration"].mean()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,7))
plt.hist(avg_song_duration, bins=30)
plt.show()
